# Lab 4: AgentCore Browser を使用した予防エージェント

## 概要
リアルタイムの AWS ドキュメント調査とプロアクティブなインフラストラクチャ推奨のために AgentCore Browser を使用する Strands ベースの予防エージェントを構築します。

## 目標
- AgentCore Browser 統合を持つ Strands エージェントを作成
- 予防重視の分析ワークフローを実装
- リアルタイムの AWS ベストプラクティス調査用ツールを構築
- インフラストラクチャ予防シナリオでエージェントをテスト
- 問題を回避するためのプロアクティブな推奨を生成

## 学習内容
- AgentCore Browser を Strands エージェントと統合する方法
- リアクティブな修復 vs 予防重視のワークフローの実装方法
- ブラウザ自動化を使用した現在の AWS ベストプラクティスの調査方法
- エージェントの予防分析と推奨パターン

## アーキテクチャ概要
```
┌─────────────────┐    ┌──────────────────────┐    ┌─────────────────────┐
│   ユーザーリクエスト │───▶│  Strands Agent       │───▶│  AgentCore Browser  │
│   （予防）        │    │  （予防）             │    │                     │
└─────────────────┘    └──────────────────────┘    └─────────────────────┘
                                │                            │
                                ▼                            ▼
                       ┌──────────────────────┐    ┌─────────────────────┐
                       │  予防分析             │    │  リアルタイム Web    │
                       │  ├─ リスク評価        │    │  調査               │
                       │  ├─ ベストプラクティス │    │  ├─ AWS ドキュメント │
                       │  └─ 推奨事項         │    │  ├─ ベストプラクティス │
                       └──────────────────────┘    │  └─ 最新標準        │
                                │                  └─────────────────────┘
                                ▼                            │
                       ┌──────────────────────┐              ▼
                       │  プロアクティブな      │    ┌─────────────────────┐
                       │  推奨事項            │◀───│  ドキュメント分析    │
                       │  （問題発生前に）      │    │  とインサイト        │
                       └──────────────────────┘    └─────────────────────┘
```

**主要コンポーネント:**
- **予防重視**: リアクティブな修正 vs プロアクティブな推奨
- **リアルタイム調査**: 現在の AWS ドキュメントへのライブアクセス
- **Browser 統合**: Web コンテンツアクセス用の AgentCore Browser
- **最新標準**: 常に最新の AWS ベストプラクティスを使用

## 0. 必要なパッケージのインストール

このセルを最初に実行して、すべての依存関係がインストールされていることを確認してください。

In [ ]:
%pip install -q -r requirements.txt
print("✅ 予防エージェントの依存関係がインストールされました")

## 1. 必要なモジュールのインポート

In [ ]:
# AWS SDK と設定
import logging
from datetime import datetime

# Strands フレームワーク
from strands import Agent
from strands.models import BedrockModel
from strands.tools import tool
from strands_tools.browser import AgentCoreBrowser

# ワークショップ設定
# Bedrock AgentCore Starter Toolkit
from bedrock_agentcore_starter_toolkit import Runtime
from lab_helpers.config import MODEL_ID, AWS_REGION, AWS_PROFILE, WORKSHOP_NAME
from lab_helpers.parameter_store import get_parameter, put_parameter
from lab_helpers.constants import PARAMETER_PATHS
from lab_helpers.lab_04.gateway_setup import AgentCoreGatewaySetup

# ノートブック用のロギング設定
logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(message)s')
logger = logging.getLogger(__name__)

print("✅ インポート完了")
print(f"   ワークショップ: {WORKSHOP_NAME}")
print(f"   リージョン: {AWS_REGION}")
print(f"   モデル: {MODEL_ID}")
print("🌐 AgentCore Browser ツールをインポートしました")
print("🔮 予防エージェントのセットアップ準備完了")

## 2. 前提条件の確認

In [ ]:
# 前提条件が利用可能か確認
try:
    # AWS 認証情報をテスト
    import boto3
    sts_client = boto3.client('sts', region_name=AWS_REGION)
    identity = sts_client.get_caller_identity()
    account_id = identity['Account']
    
    # AgentCore Browser の可用性をテスト
    browser_tool = AgentCoreBrowser(region=AWS_REGION)
    
    print(f"✅ 前提条件を確認しました: AWS アカウント {account_id}、AgentCore Browser 利用可能")
    print(f"   リージョン: {AWS_REGION}")
    print(f"   プロファイル: {AWS_PROFILE}")
    print(f"   モデル ID: {MODEL_ID}")
    print(f"   ID: {identity.get('Arn', 'Unknown')}")
    print(f"   Browser ツール: AgentCoreBrowser 初期化完了")
    
except Exception as e:
    print(f"❌ エラー: {e}")
    print("AWS 認証情報が設定されており、AgentCore Browser の権限があることを確認してください。")

## 3. Browser 統合を持つ予防エージェントの作成

**目標:** リアルタイム Web 調査のための AgentCore Browser を持つ Strands エージェントをセットアップします。

**アプローチ:** Browser ツールを初期化し、予防重視のシステムプロンプトでエージェントを作成します。

**重要な学習ポイント:** AgentCore Browser が予防のためのリアルタイムドキュメント調査をどのように実現するか。

In [ ]:
### 3.1: AgentCore Browser ツールで予防エージェントをセットアップ

def setup_prevention_agent(region=AWS_REGION):
    """AgentCore Browser ツール付きの Strands エージェントをセットアップ"""
    try:
        # Browser ツールを初期化 - strands_tools.browser パターンに従う
        browser_tool = AgentCoreBrowser(region=region)
        
        # Bedrock モデルをセットアップ
        model = BedrockModel(
            model_id=MODEL_ID,
            streaming=True,
        )
        
        # Browser ツール付きのエージェントを作成 
        agent = Agent(
            model=model, 
            tools=[browser_tool.browser]
        )
        
        logger.info("✅ SRE 予防エージェント準備完了（AgentCore Browser 付き）")
        logger.info(f"🌍 リージョン: {region}")
        logger.info("🌐 Browser ツール: AgentCoreBrowser 統合済み")
        
        return agent
        
    except Exception as e:
        logger.error(f"❌ エージェントのセットアップに失敗しました: {e}")
        return None

# エージェントをセットアップ
agent = setup_prevention_agent()
if agent:
    print("✅ Strands 予防エージェントが正常に作成されました")
    print(f"   モデル: {MODEL_ID}")
    print(f"   ツール: 1（AgentCore Browser）")
    print("🌐 AgentCore Browser 統合準備完了")
    print("🔮 予防分析の準備完了")
else:
    print("❌ エージェントのセットアップに失敗しました！")
    print("   AgentCore Browser の初期化と AWS 認証情報を確認してください")

## 4. Browser 統合のテスト

**目標:** 基本的な Web アクセステストで AgentCore Browser の機能を確認します。

**アプローチ:** シンプルな AWS ホームページアクセスで Browser ツールをテストします。

**重要な学習ポイント:** Browser の接続性とコンテンツ抽出の検証方法。

In [ ]:
### 4.1: Browser 統合のテスト

# エージェントをセットアップ

agent = setup_prevention_agent()

def test_browser_integration(agent):
    """AgentCore Browser 統合をテスト"""
    if not agent:
        print("❌ テスト用のエージェントが利用できません")
        return False
    
    print("🧪 AgentCore Browser 統合をテスト中...")
    print("=" * 50)
    
    test_prompt = """
    Please use the browser tool to visit https://aws.amazon.com and get the page title
    
    
    This is just a test to verify the browser tool is working correctly.
    """
    
    try:
        start_time = datetime.now()
        response = agent(test_prompt)
        test_time = (datetime.now() - start_time).total_seconds()
        
        print(f"✅ Browser テストが {test_time:.2f} 秒で完了しました")
        
        # レスポンスを表示（長すぎる場合は切り詰め）
        response_content = response.message.get('content', [])
        if response_content:
            for content in response_content:
                if isinstance(content, dict) and 'text' in content:
                    text = content['text']
                    if len(text) > 800:
                        print("\n📋 BROWSER テスト結果（最初の800文字）:")
                        print("-" * 30)
                        print(f"{text[:800]}...\n\n[フル出力: {len(text)} 文字]")
                    else:
                        print("\n📋 BROWSER テスト結果:")
                        print("-" * 30)
                        print(text)
                elif isinstance(content, str):
                    print("\n📋 BROWSER テスト結果:")
                    print("-" * 30)
                    print(content)
        
        return True
        
    except Exception as e:
        print(f"❌ Browser テストに失敗しました: {e}")
        return False

# Browser テストを実行
if agent:
    print("💡 Browser テストを実行するには、以下の行のコメントを解除してください:")
    test_result = test_browser_integration(agent)
    if test_result:
        print("\n🎉 AgentCore Browser 統合テスト成功！")
    else:
        print("\n❌ AgentCore Browser 統合テスト失敗！")
    print("⚠️  注意: これは AWS ホームページへの実際の Web リクエストを行います")
else:
    print("❌ Browser テスト用のエージェントが利用できません")

## Memory からコンテキストを補強

診断情報でコンテキストを補強するために、キュレーションされた Memory から追加情報を取得しましょう。

In [ ]:
agent_memory_client = boto3.client("bedrock-agentcore", region_name=AWS_REGION)

memory_id = get_parameter(PARAMETER_PATHS['memory']['memory_id'])
memory_session_id = get_parameter(PARAMETER_PATHS['memory']['default_session_id'])

print(memory_id)
print(memory_session_id)
actor_id='diagnostics_agent'

  
# エージェント Memory に追加されたイベントをリスト（書き込み成功を確認）
params = {
                "memoryId": memory_id,
                "actorId": actor_id,
                "sessionId": memory_session_id,
                "includePayloads": True
            }
# すべてのメッセージを取得
response = agent_memory_client.list_events(**params)
additional_context=""
for event in response.get("events", []):
    payload = event.get('payload', [])
    for i, item in enumerate(payload):
        if 'conversational' in item:
            text = item['conversational']['content']['text']
            additional_context+=text
additional_context

## 5. 予防分析ワークフロー

**目標:** 実際の AWS ドキュメント調査を使用した予防分析ワークフローをデモンストレーションします。

**アプローチ:** Browser を使用してベストプラクティスを調査する包括的な予防分析を実行します。

**重要な学習ポイント:** 予防エージェントがプロアクティブな対策をどのように調査し推奨するか。

In [ ]:
### 5.1: 予防分析を実行

# エージェントをセットアップ

agent = setup_prevention_agent()

def analyze_prevention_opportunities(agent):
    """インフラストラクチャの予防機会を分析"""
    if not agent:
        logger.error("❌ エージェントが初期化されていません")
        return None
    
    logger.info("🔮 予防機会の分析を開始...")
    
    prevention_prompt = f"""
    I need you to analyze our CRM infrastructure for prevention opportunities using the browser tool to access AWS documentation. 
    Our CRM application is hosted on EC2 instances with a DynamoDB backend for customer data. 

    Here is additional information about the issues identified and fixes that have been applied:

    {additional_context}
    
    Please use the browser tool to access these specific AWS documentation pages and provide analysis:
    
    1. First, use the browser tool to visit: https://docs.aws.amazon.com/amazondynamodb/latest/developerguide/best-practices.html
    2. Then visit: https://docs.aws.amazon.com/AWSEC2/latest/UserGuide/ec2-best-practices.html  
    3. Finally visit: https://docs.aws.amazon.com/wellarchitected/latest/framework/
    
    Based on what you find in the AWS documentation, provide analysis focusing on:
    
    1. **Proactive Infrastructure Management**: Best practices we should implement
    2. **Customer Impact Prevention**: How to prevent issues that could affect customer experience  
    3. **Performance Optimization**: Latest AWS recommendations for EC2 and DynamoDB optimization
    4. **Monitoring and Alerting**: Best practices for proactive monitoring
    
    Provide your analysis with:
    - Executive summary of prevention opportunities
    - Customer impact prioritization  
    - Implementation roadmap with AWS best practices
    - Success metrics for measuring prevention effectiveness
    
    Use the browser tool to get the most current AWS documentation and recommendations.
    """
    
    try:
        start_time = datetime.now()
        response = agent(prevention_prompt)
        analysis_time = (datetime.now() - start_time).total_seconds()
        
        logger.info(f"✅ 予防分析が {analysis_time:.2f} 秒で完了しました")
        
        return {
            'analysis_time': analysis_time,
            'response': response,
            'timestamp': datetime.now().isoformat()
        }
        
    except Exception as e:
        logger.error(f"❌ 予防分析に失敗しました: {e}")
        return None

# 完全な予防分析を実行
if agent:
    print("🚀 完全な予防分析ワークフローを開始...")
    print("=" * 60)
    print("🔮 Browser を使用して現在の AWS ベストプラクティスを調査します")
    print("📚 最新の推奨事項のためにライブの AWS ドキュメントにアクセスします")
    print("⏳ 複数のドキュメントページをブラウジングするため、数分かかる場合があります...")
    result = analyze_prevention_opportunities(agent)
    
    if result:
        print(f"\n🎯 予防分析結果:")
        print(f"分析時間: {result['analysis_time']:.2f} 秒")
        print(f"タイムスタンプ: {result['timestamp']}")
         
         # エージェントのレスポンスを表示
        response_content = result['response'].message.get('content', [])
        if response_content:
            for content in response_content:
                if isinstance(content, dict) and 'text' in content:
                    text = content['text']
                    if len(text) > 2000:
                        print(f"\n📋 予防分析（最初の2000文字）:\n{text[:2000]}...")
                    else:
                        print(f"\n📋 予防分析:\n{text}")
                elif isinstance(content, str):
                    print(f"\n📋 予防分析:\n{content}")
         
        print("\n🎉 予防分析ワークフローが正常に完了しました！")
    else:
        print("❌ 予防分析に失敗しました！")
    
    print("⚠️  注意: これは複数の Browser リクエストを行うため、数分かかる場合があります")
    print("🔒 これはリアクティブな修復 vs プロアクティブな分析を示しています")
    
else:
    print("❌ 予防分析用のエージェントが利用できません！")

## 6. AgentCore Runtime へのデプロイ

**目標:** サーバーレス実行のために予防エージェントを Amazon Bedrock AgentCore Runtime にデプロイします。

**アプローチ:** AgentCoreRuntimeDeployer を使用して Runtime インフラストラクチャを作成し、エージェントをデプロイします。

**重要な学習ポイント:** Browser 統合を持つ Strands エージェントを本番環境対応のサーバーレスインフラストラクチャにデプロイする方法。

### 6.1: Runtime Deployer の初期化と IAM ロールの作成

In [ ]:
from lab_helpers.lab_04 import AgentCoreRuntimeDeployer, store_runtime_configuration

# Deployer を初期化
deployer = AgentCoreRuntimeDeployer(
    region=AWS_REGION,
    prefix=WORKSHOP_NAME,
    verbose=True
)

# 前提条件を確認
print("✓ 前提条件を確認中...")
if deployer.check_prerequisites():
    print("✅ すべての前提条件を満たしています")
else:
    print("❌ 前提条件を満たしていません")
    print("以下をインストールしてください: pip install fastmcp bedrock-agentcore-starter-toolkit strands-agents-tools")

# Browser 権限付きの IAM ロールを作成
print("\n🔐 Runtime 用の IAM ロールを作成中...")
role_info = deployer.create_runtime_iam_role()
print(f"   ロール ARN: {role_info['role_arn']}")

### 6.2: AgentCore Runtime へのデプロイ

In [ ]:
### 6.2 a: デプロイ用のエージェントコード

# エージェントコードファイルを読み込み
with open('lab_helpers/lab_04/runtime_mcp_agent_code.py', 'r') as f:
    agentcore_agent_code = f.read()

print("✅ エージェントコードを読み込みました: lab_helpers/lab_04/runtime_mcp_agent_code.py")
print(f"✓ エージェントコードを読み込みました: {len(agentcore_agent_code)} バイト")
print(f"✓ コードサイズ: {len(agentcore_agent_code) / 1024:.1f} KB")
print(f"✓ AWS_REGION: {AWS_REGION}") 

In [ ]:
### 6.2 b: エージェントコードをディスクに書き込み
with open('agent-prevention.py', 'w') as f:
      f.write(agentcore_agent_code)

print(f"✅ エージェントコードを書き込みました: agent-prevention.py")

### 6.3 JWT Authorizer で Runtime を設定

**runtime.configure() の機能:**
- エージェントコードと依存関係を検証
- Dockerfile と AWS 設定ファイルを生成
- デプロイメントブループリントを準備（ローカル操作、AWS リソースはまだ作成されません）
- 実行ロールとトークン検証をセットアップ

**JWT Authorizer の設定:**
`authorizer_configuration` パラメータは、Runtime が受信トークンを検証する方法を指示します:
- **discoveryUrl**: Runtime が署名検証用の公開鍵を取得する Cognito OIDC エンドポイント
- **allowedClients**: User Auth クライアント（直接ユーザー）と M2M クライアント（Gateway）の両方が許可されます

**自動的な Runtime の動作:**
```
リクエストの Bearer トークン → 署名を検証 → 発行者を確認（Cognito）→ allowedClients でクライアント ID を検証 → 許可または拒否
```

In [ ]:
### 6.3 a: Lab-01 の SSM Parameter Store から Cognito 設定を取得
from lab_helpers.parameter_store import get_parameter
from lab_helpers.constants import PARAMETER_PATHS

# Lab-01 の SSM Parameter Store から Cognito 設定を取得
cognito_domain = get_parameter(PARAMETER_PATHS['cognito']['domain'])
user_pool_id = get_parameter(PARAMETER_PATHS['cognito']['user_pool_id'])
m2m_client_id = get_parameter(PARAMETER_PATHS['cognito']['m2m_client_id'])
user_auth_client_id = get_parameter(PARAMETER_PATHS['cognito']['user_auth_client_id'])

# Cognito Discovery URL を構築 - Runtime がトークン検証用の公開鍵を取得するために使用
# Parameter Store から取得した Cognito ドメインを使用（手動構築よりも信頼性が高い）
discovery_url = f"https://cognito-idp.{AWS_REGION}.amazonaws.com/{user_pool_id}/.well-known/openid-configuration"

print(f"✅ Cognito 設定を取得しました")
print(f"   Discovery URL: {discovery_url}")
print(f"   許可されたクライアント: User Auth + M2M")
print(f"   User Auth クライアント ID: {user_auth_client_id}")
print(f"   M2M クライアント ID: {m2m_client_id}")

In [ ]:
### 6.3 b: JWT Authorizer で Runtime を設定

# Runtime オブジェクトを初期化
runtime = Runtime()

# JWT Authorizer 設定
# - discoveryUrl: Cognito OIDC エンドポイント（Runtime が自動的に公開鍵を取得）
# - allowedClients: User Auth と M2M の両方のクライアントが Runtime を呼び出し可能
authorizer_config = {
    "customJWTAuthorizer": {
        "discoveryUrl": discovery_url,
        "allowedClients": [user_auth_client_id, m2m_client_id]
    }
}

# 重要: JWT トークン検証で Runtime を設定
runtime.configure(
      entrypoint='agent-prevention.py',
      execution_role=role_info['role_arn'],
      auto_create_ecr=True,
      requirements_file='requirements.txt',
      region=AWS_REGION,
      agent_name=f"{WORKSHOP_NAME}_prevention_runtime",
      protocol="MCP",
      authorizer_configuration=authorizer_config  # ← トークン検証を有効化
)

print("✅ JWT Authorizer で Runtime を設定しました")
print(f"   プロトコル: MCP")
print(f"   JWT トークン検証: 有効")
print(f"   許可されたトークン: User Auth + M2M")

### 6.4: AgentCore に Runtime を起動

Python SDK の `runtime.launch()` を使用して、設定した Runtime を AgentCore にデプロイします。この重要なステップで、ローカルエージェントを本番サーバーレスサービスに変換します。

#### 6.4 a: 起動プロセスの理解

  **runtime.launch() 実行中に起こること:**

  1️⃣ CodeBuild が Docker コンテナのビルドを開始

  2️⃣ requirements.txt の依存関係がインストールされる

  3️⃣ イメージが Amazon ECR にプッシュされる（自動作成）

  4️⃣ AgentCore が Runtime を MCP サービスとして登録
  
  5️⃣ CloudWatch ログが設定される

  ⏱️ **典型的な所要時間:** 5-10分

In [ ]:
# すべてのセルを一括実行するシナリオに対応するために追加
import time
time.sleep(10)

In [ ]:
### 6.4 a: runtime.launch() を実行

print("\n🚀 AgentCore に Runtime を起動中...\n")

try:
    # 競合時に自動更新で Runtime を起動
    # これは同期処理 - CodeBuild と初期ヘルスチェックを待機
    launch_result = runtime.launch(auto_update_on_conflict=True)
    
    # デプロイメント ARN を抽出
    runtime_arn = launch_result.agent_arn
    
    print(f"✅ Runtime が正常に起動しました！")
    print(f"   Runtime ARN: {runtime_arn}")
    print(f"\n📝 次のセクション用に設定を保存しました:")
    
except Exception as e:   
    print(f"❌ 起動に失敗しました: {e}")
    print(f"\nトラブルシューティング:")
    print(f"  • CodeBuild のサービス制限を確認")
    print(f"  • IAM ロールの ECR 権限を確認")
    print(f"  • CloudWatch ログでビルドエラーを確認")
    print(f"  • requirements.txt のすべての依存関係が正しいことを確認")
    raise

In [ ]:
### 6.4 b: Runtime 設定を保存

# Runtime 設定を抽出して保存
runtime_arn = launch_result.agent_arn
runtime_id = getattr(launch_result, 'agent_id', None)

# 設定を保存
from lab_helpers.lab_04 import store_runtime_configuration
store_runtime_configuration(runtime_arn, runtime_id, region=AWS_REGION, prefix=WORKSHOP_NAME)

print("\n✅ Runtime がデプロイされ、設定されました")
print(f"   ARN: {runtime_arn}")
print(f"   Gateway 登録の準備完了")

## 7. CloudWatch ログの設定

**目標:** Runtime コンテナログ用の CloudWatch Logs Delivery をセットアップします。

**アプローチ:** configure_runtime_logging を使用してログ配信を自動的にセットアップします。

**重要な学習ポイント:** AgentCore がオブザーバビリティのために CloudWatch とどのように統合されるか。

In [ ]:
from lab_helpers.lab_04.configure_logging import configure_runtime_logging

# CloudWatch Logs Delivery を設定
print("📊 CloudWatch Logs Delivery を設定中...")

logging_config = configure_runtime_logging(
    runtime_arn=runtime_arn,
    runtime_id=runtime_id,
    region=AWS_REGION,
    log_type="APPLICATION_LOGS"
)

print(f"\n✅ ロギングを設定しました:")
print(f"   ロググループ: {logging_config['log_group_name']}")
print(f"   ステータス: {logging_config['delivery_status']}")

## 8. AgentCore Gateway のデプロイ

**目標:** Gateway インフラストラクチャを作成し、Runtime をターゲットとして登録します。

**アプローチ:** Cognito JWT 認証付きの Gateway を作成し、OAuth2 M2M 認証情報で Runtime を登録します。

**重要な学習ポイント:** デュアルレイヤー認証（受信は JWT、送信は OAuth2 M2M）のセットアップ方法。

In [ ]:
# Lab-01 から Cognito 認証情報を取得
user_auth_client_id = get_parameter(PARAMETER_PATHS['cognito']['user_auth_client_id'])
user_pool_id = get_parameter(PARAMETER_PATHS['cognito']['user_pool_id'])

# Cognito OIDC Discovery URL を構築
discovery_url = f"https://cognito-idp.{AWS_REGION}.amazonaws.com/{user_pool_id}/.well-known/openid-configuration"

# ヘルパーを使用して Gateway IAM ロールを作成 
gateway_setup = AgentCoreGatewaySetup(region=AWS_REGION, prefix=WORKSHOP_NAME, verbose=False)
role_info = gateway_setup.create_gateway_service_role()
role_arn = role_info['role_arn']

# boto3 で Gateway を直接作成（シンプルな API 呼び出し）
agentcore = boto3.client('bedrock-agentcore-control', region_name=AWS_REGION)

gateway_response = agentcore.create_gateway(
      name=f"aiml301-prevention-gateway",
      roleArn=role_arn,
      protocolType="MCP",
      authorizerType="CUSTOM_JWT",
      authorizerConfiguration={
          "customJWTAuthorizer": {
              "discoveryUrl": discovery_url,
              "allowedClients": [user_auth_client_id]
          }
      }
  )

gateway_id = gateway_response['gatewayId']
gateway_url = gateway_response['gatewayUrl']

# 設定を保存
put_parameter(PARAMETER_PATHS['lab_04']['gateway_id'], gateway_id)
put_parameter(PARAMETER_PATHS['lab_04']['gateway_role_arn'], role_arn)

print("✅ JWT 認可付きで Gateway をデプロイしました")
print(f"   Gateway ID: {gateway_id}")
print(f"   Gateway URL: {gateway_url}")
print(f"   認証タイプ: Cognito JWT")

### 8.1: M2M 認証で Runtime を Gateway ターゲットとして追加

  **目標:** OAuth2 M2M 認証で Runtime を Gateway ターゲットとして登録します。

  **アーキテクチャ:**

```
  User（JWT）→ Gateway（JWT 検証）→ Runtime（M2M トークン）
                      ↓
              Gateway が自動的に:
              - GetResourceOauth2Token を呼び出し
              - Secrets Manager から認証情報を取得
              - Cognito から M2M アクセストークンを取得
              - リクエストに Bearer トークンを注入
                      ↓
              Authorization: Bearer {token} でRuntime を呼び出し
```
  **Gateway が自動的に M2M OAuth2 トークンを送信する方法:**

  1. **認証情報プロバイダーストレージ**
     - OAuth2 認証情報プロバイダーが `clientId` + `clientSecret` を AWS Secrets Manager に保存（暗号化）
     - プロバイダー ARN がこのセキュアなストレージ場所を指す

  2. **自動トークン取得フロー**
  ```
     credentialProviderConfigurations で Gateway ターゲットを作成
         ↓
     Gateway が Runtime ターゲットを呼び出す必要がある
         ↓
     Gateway が GetResourceOauth2Token API を呼び出し（自動、組み込み）
         ↓
     AgentCore Identity が Secrets Manager から client_id + client_secret を取得
         ↓
     AgentCore が Cognito トークンエンドポイントを呼び出し:
     POST /token
     Body: grant_type=client_credentials&client_id=...&client_secret=...&scope=...
         ↓
     Cognito が M2M アクセストークンを返す
         ↓
     AgentCore がトークンをキャッシュし、更新ライフサイクルを管理
         ↓
     Gateway がリクエストヘッダーにトークンを注入:
     Authorization: Bearer {access_token}
         ↓
     Gateway が Bearer トークン付きで Runtime MCP エンドポイントを呼び出し
         ↓
     Runtime が Cognito 公開鍵（JWKS）を使用して JWT 署名を検証
```
  3. **カスタムコード不要**
  - すべての認証情報管理が自動
  - トークン更新が自動
  - トークン注入が自動
  - コードでは認証情報プロバイダー ARN を指定するだけ

  **重要な学習ポイント:** デュアル認証 - ユーザーベースの受信（JWT）、サービスベースの送信（OAuth2 M2M）。

In [ ]:
# すべてのセルを一括実行するシナリオに対応するために追加
import time
time.sleep(10)

Lab-03 を正常に実行した場合、そこで作成した認証情報プロバイダーを再利用します。Lab-04 を直接実行している場合は、以下のコメントを解除してください

In [ ]:
### 8.1 セキュアアクセス用の AgentCore Identity CredentialsProvider を作成
## Lab-01 の Cognito セットアップから M2M 認証情報を取得
#m2m_client_id = get_parameter(PARAMETER_PATHS['cognito']['m2m_client_id'])
#m2m_client_secret = get_parameter(PARAMETER_PATHS['cognito']['m2m_client_secret'])
#user_pool_id = get_parameter(PARAMETER_PATHS['cognito']['user_pool_id'])

## Cognito OIDC Discovery URL を構築
#discovery_url = f"https://cognito-idp.{AWS_REGION}.amazonaws.com/{user_pool_id}/.well-known/openid-configuration"

# AgentCore クライアントを初期化
#agentcore = boto3.client('bedrock-agentcore-control', region_name=AWS_REGION)

## OAuth2 認証情報プロバイダーを作成
## M2M 認証情報を Secrets Manager に安全に保存
#credential_provider_response = agentcore.create_oauth2_credential_provider(
#      name='aiml301-m2m-credentials',
#      credentialProviderVendor='CustomOauth2',
#      oauth2ProviderConfigInput={
#          'customOauth2ProviderConfig': {
#              'clientId': m2m_client_id,
#              'clientSecret': m2m_client_secret,
#              'oauthDiscovery': {
#                  'discoveryUrl': discovery_url
#              }
#          }
#      }
#  )

## レスポンスを抽出
#oauth2_provider_arn = credential_provider_response['credentialProviderArn']
#client_secret_arn = credential_provider_response['clientSecretArn']['secretArn']

## 次のセクション用に SSM に保存
#put_parameter(PARAMETER_PATHS['lab_03']['oauth2_provider_arn'], oauth2_provider_arn)
#put_parameter(PARAMETER_PATHS['lab_03']['oauth2_secret_arn'], client_secret_arn)

#print("✅ OAuth2 認証情報プロバイダーを作成しました")
#print(f"\n📋 認証情報ストレージ:")
#print(f"   プロバイダー ARN: {oauth2_provider_arn}")
#print(f"   シークレット ARN: {client_secret_arn}")
#print(f"   保存場所: AWS Secrets Manager（暗号化）")
#print(f"   認証情報: M2M client_id + client_secret")

### 8.2: OAuth2 認証情報プロバイダーを作成し Runtime ターゲットを登録

In [ ]:
# Parameter Store から M2M 認証情報を取得
# SSM から設定を取得  
gateway_id = get_parameter(PARAMETER_PATHS['lab_04']['gateway_id'], region_name=AWS_REGION)
runtime_arn = get_parameter(PARAMETER_PATHS['lab_04']['runtime_arn'], region_name=AWS_REGION)

# Lab-03 で作成した認証情報プロバイダーを使用する場合
oauth2_provider_arn = get_parameter(PARAMETER_PATHS['lab_03']['oauth2_provider_arn'], region_name=AWS_REGION)
# Lab-03 を使用しない場合は、以下のコメントを解除して新しい認証情報プロバイダーを作成
#oauth2_provider_arn = get_parameter(PARAMETER_PATHS['lab_04']['oauth2_provider_arn'], region_name=AWS_REGION)

resource_server_id = get_parameter(PARAMETER_PATHS['cognito']['resource_server_identifier'], region_name=AWS_REGION)

In [ ]:
print(f"   Gateway ID: {gateway_id}")
print(f"   Gateway URL: {gateway_url}")
print(f"   Runtime ARN: {runtime_arn}")

In [ ]:
### M2M OAuth2 で Runtime のターゲットを作成

import urllib.parse

# 重要: Runtime ARN からエンドポイント URL を構築
# フォーマット: https://bedrock-agentcore.{region}.amazonaws.com/runtimes/{URL_ENCODED_ARN}/invocations?qualifier=DEFAULT
# 例の ARN: arn:aws:bedrock-agentcore:us-west-2:123456789012:runtime/my-runtime
encoded_arn = urllib.parse.quote(runtime_arn, safe='')
endpoint_url = f"https://bedrock-agentcore.{AWS_REGION}.amazonaws.com/runtimes/{encoded_arn}/invocations?qualifier=DEFAULT"

# きめ細かい認可用の M2M スコープ
m2m_scopes = [
    f"{resource_server_id}/mcp.invoke",
    f"{resource_server_id}/runtime.access"
]

# AgentCore クライアントを初期化
agentcore = boto3.client('bedrock-agentcore-control', region_name=AWS_REGION)

# Gateway ターゲットを作成 - M2M OAuth2 付きの外部 MCP サーバーとしての Runtime
# Gateway は自動的に:
# 1. oauth2_provider_arn を使用して Cognito から M2M トークンを取得
# 2. Runtime 呼び出し時に Bearer トークンを含める
# 3. Runtime がトークンを検証し、スコープに基づいて操作を認可
target_response = agentcore.create_gateway_target(
    gatewayIdentifier=gateway_id,
    name="aiml301-runtime-target",
    description="M2M OAuth2 認証付きの AgentCore Runtime",
    targetConfiguration={
        "mcp": {
            "mcpServer": {
                "endpoint": endpoint_url  # ← URL エンコードされた Runtime ARN から構築
            }
        }
    },
    credentialProviderConfigurations=[
        {
            "credentialProviderType": "OAUTH",
            "credentialProvider": {
                "oauthCredentialProvider": {
                    "providerArn": oauth2_provider_arn,  # ← セクション 9.1 の OAuth2 認証情報プロバイダーを参照
                    "scopes": m2m_scopes
                }
            }
        }
    ]
)

target_id = target_response['targetId']
put_parameter(PARAMETER_PATHS['lab_04']['gateway_runtime_target'], target_id, region_name=AWS_REGION)

print("✅ M2M OAuth2 で Runtime を Gateway ターゲットとして追加しました")
print(f"   ターゲット ID: {target_id}")
print(f"   Runtime ARN: {runtime_arn}")
print(f"   エンドポイント: {endpoint_url}")
print(f"   認証情報プロバイダー: {oauth2_provider_arn}")

In [ ]:
import time

### 9.2 ターゲットステータスを確認して同期

# ターゲットが READY になるまで待機
print("\n⏳ ターゲットが READY になるのを待機中...")
for attempt in range(30):
      target_info = agentcore.get_gateway_target(
          gatewayIdentifier=gateway_id,
          targetId=target_id
      )
      status = target_info.get('status', 'UNKNOWN')

      if status == 'READY':
          print(f"✅ ターゲット準備完了")
          break
      if status == 'FAILED' or status == 'SYNCHRONIZE_UNSUCCESSFUL':
          print(f"❌ ターゲットがエラー状態: {target_info.get('statusReasons', 'エラーメッセージなし')}")
          break
      time.sleep(5)

# ツールを検出するために同期
agentcore.synchronize_gateway_targets(
      gatewayIdentifier=gateway_id,
      targetIdList=[target_id]
  )

print("\n✅ 完了 - Gateway が M2M トークンを自動的に管理します")

  ##### 重要な注意: 
  Gateway ロールに必要な権限:

  - bedrock-agentcore:GetResourceOauth2Token - 認証情報プロバイダーから M2M トークンを取得するため
  - secretsmanager:GetSecretValue - 保存された OAuth2 認証情報にアクセスするため

  なぜ？ Gateway が Runtime への MCP 接続を開始し、OAuth2 M2M 認証情報で認証する必要があるため。

  Runtime ロールに必要な権限:

  - bedrock-agentcore:GetWorkloadAccessToken - ワークロード ID を検証するため
  - bedrock-agentcore:CreateWorkloadIdentity - 自身の ID を確立するため
  - bedrock-agentcore:GetResourceOauth2Token - Gateway からの受信 OAuth2 トークンを検証するため

  なぜ？ Runtime は OAuth2 Bearer トークンを含む Gateway からの MCP リクエストを受信します。Runtime はこれらのトークンを検証し、受信 MCP 呼び出しを認証する必要があります。

## 9. MCP クライアント経由でデプロイされたエージェントをテスト

**目標:** JWT 認証で Gateway に接続してデプロイされたエージェントが動作することを確認します。

**アプローチ:** Cognito で認証し、MCPClient を Gateway に接続し、予防ツールを呼び出します。

**重要な学習ポイント:** MCP プロトコルを使用した本番エージェントのテスト方法。

### 9.1: Cognito で認証して JWT トークンを取得

In [ ]:
import boto3
from urllib.parse import urlencode
import requests
import json

# テストユーザーの認証情報を取得
test_user_email = get_parameter(PARAMETER_PATHS['cognito']['test_user_email'])
test_user_password = get_parameter(PARAMETER_PATHS['cognito']['test_user_password'])

print("🔐 Cognito で認証中...")
print(f"   ユーザー: {test_user_email}")

cognito_client = boto3.client('cognito-idp', region_name=AWS_REGION)

# ユーザーを認証
auth_response = cognito_client.initiate_auth(
    ClientId=user_auth_client_id,
    AuthFlow='USER_PASSWORD_AUTH',
    AuthParameters={
        'USERNAME': test_user_email,
        'PASSWORD': test_user_password
    }
)

access_token = auth_response['AuthenticationResult']['AccessToken']
print(f"✅ JWT トークンを取得しました")
print(f"   トークン長: {len(access_token)} 文字")

In [ ]:
print((access_token))

### 9.2: MCPClient に接続してツールをリスト

In [ ]:
from lab_helpers.lab_04.mcp_client import MCPClient

# MCP クライアントを Gateway に接続
print("🔌 MCPClient 経由で Gateway に接続中...")
client = MCPClient(gateway_url, access_token, timeout=300)

# MCP セッションを初期化
print("\n📡 MCP セッションを初期化中...")
server_info = client.initialize()

# 利用可能なツールをリスト
print("\n🔧 利用可能なツールをリスト中...")
tools = client.list_tools()

print(f"\n✅ {len(tools)} 個のツールが見つかりました:")
for i, tool in enumerate(tools, 1):
    print(f"   {i}. {tool['name']}")
    desc = tool.get('description', '説明なし')
    first_line = desc.split('\n')[0][:80]
    print(f"      {first_line}...")

In [ ]:
# すべてのセルを一括実行するシナリオに対応するために追加
import time
time.sleep(10)

### 9.3: 予防分析ツールの呼び出し

In [ ]:
# analyze_infrastructure_prevention ツールを呼び出し
print("🔮 prevention_agent ツールを呼び出し中...")

result = client.call_tool(
    "aiml301-runtime-target___research_agent",
    {
        "research_topic_query": f"""この{additional_context}に基づいて、将来の問題を最小限に抑えるためのベストプラクティスを調査してください。"""
    }
)

print(f"\n✅ 予防分析完了:")
if result['isError'] == False:

    print(result['content'][0]['text'])
    
else:
    print(f"   エラー: {result}")

## 10. クリーンアップ

**目標:** すべての Lab-04 AWS リソースを削除します。

**アプローチ:** cleanup_lab_04 を使用して Gateway、Runtime、IAM ロール、ログを削除します。

**注意:** 継続的な課金を避けるため、テストが完了したらこれを実行してください。

In [ ]:
from lab_helpers.lab_04 import cleanup_lab_04

# すべての Lab-04 リソースをクリーンアップ
print("🧹 Lab-04 リソースをクリーンアップ中...")
print("\n警告: 以下が削除されます:")
print("  • AgentCore Gateway とすべてのターゲット")
print("  • AgentCore Runtime")
print("  • OAuth2 認証情報プロバイダー")
print("  • IAM ロール")
print("  • CloudWatch ログ")
print("\n続行するには、以下の行のコメントを解除してください:")
print("# cleanup_lab_04(region_name=AWS_REGION, verbose=True)")

# クリーンアップを実行するにはコメントを解除:
#cleanup_lab_04(region_name='us-west-2', verbose=True)

## まとめ: Lab 4 - 本番デプロイ付き予防エージェント

✅ **完了:**
1. ✓ Browser 統合を持つ Strands 予防エージェントを作成
2. ✓ Browser の Web アクセスとコンテンツ抽出をテスト
3. ✓ 予防分析ワークフローを実行（ローカル）
4. ✓ エージェントを AgentCore Runtime にデプロイ
5. ✓ CloudWatch Logs Delivery を設定
6. ✓ Cognito JWT 認証付き Gateway をデプロイ
7. ✓ OAuth2 M2M で Runtime を MCP ターゲットとして登録
8. ✓ JWT 認証で MCPClient 経由でテスト

**アーキテクチャ:**
```
予防エージェント（Strands + Browser）
    ↓
AgentCore Runtime（FastMCP）
    ↓
Gateway（受信 JWT 認証、送信 OAuth2 M2M）
    ↓
MCPClient（JWT 経由でテスト）
```

**主要機能:**
- **プロアクティブな予防**: インフラストラクチャの予防機会を分析
- **調査ベース**: 現在の AWS ベストプラクティスのために AgentCore Browser を使用
- **本番環境対応**: サーバーレス、自動スケーリング、オブザーバブル
- **セキュア**: デュアルレイヤー認証（JWT + OAuth2）

**利用可能な 3 つの MCP ツール:**
1. `analyze_infrastructure_prevention` - 予防機会を調査
2. `research_aws_best_practices` - AWS トピックを調査
3. `validate_prevention_environment` - 環境の準備状況を検証

**次のステップ:**
- Lab-05: マルチエージェントオーケストレーション（修復 + 予防）
- Lab-03 + Lab-04 を統合された SRE 自動化パイプラインに結合
- 調査 → 予防 → 修復ワークフロー